In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [2]:
dataName = 'WCup2022_TeamData.csv'
DFTeamData = pd.read_csv(dataName)
DFTeamData

,Team,Group,Area,Ranking
0,Qatar,A,Host,50
1,Ecuador,A,SouthAmerica,44
2,Senegal,A,Africa,18
3,Netherlands,A,Europe,8
4,England,B,Europe,5
5,Iran,B,Asia,20
6,USA,B,Caribbean,16
7,Wales,B,Europe,19
8,Argentina,C,SouthAmerica,3
9,SaudiArabia,C,Asia,51


In [3]:
def Ranking(RoundRobin):
    Team_list = RoundRobin.index.tolist()
    
    for team_index in Team_list:
        scored_sum = 0
        conceded_sum = 0
        winningPoint = 0
        for team_columns in Team_list:
            score = RoundRobin.at[team_index, team_columns]
            if type(score) is float:
                continue
            score_list = score.split(',')
            if len(score_list) != 2:
                continue
            
            scored, conceded = int(score_list[0]), int(score_list[1])
            scored_sum += scored
            conceded_sum += conceded
            if scored > conceded: winningPoint += 3
            elif scored == conceded: winningPoint += 1
                
        RoundRobin.at[team_index, 'WinningPoints'] = winningPoint
        RoundRobin.at[team_index, 'Score'] = scored_sum
        RoundRobin.at[team_index, 'Conceded'] = conceded_sum
        RoundRobin.at[team_index, 'GoalDiff'] = scored_sum - conceded_sum
        
    WinningPoints_rank = RoundRobin['WinningPoints'].rank(ascending=False, method='min')
    WinningPoints_list = WinningPoints_rank.tolist()
    RoundRobin['Ranking'] = WinningPoints_list
    Duplication_list1 = []
    for i in range(1,5):
        if WinningPoints_list.count(i) >= 2:
            Duplication_list1.append(i)
    for duplication1 in Duplication_list1:
        RoundRobin_duplication1 = RoundRobin.loc[RoundRobin['Ranking'] == duplication1]
        GoalDiff_rank = RoundRobin_duplication1['GoalDiff'].rank(ascending=False, method='min')
        GoalDiff_list = GoalDiff_rank.tolist()
        GoalDiff_index = GoalDiff_rank.index.tolist()
        for j in range(1,len(GoalDiff_list)+1):
            if GoalDiff_list.count(j) == 1:
                RoundRobin.at[GoalDiff_index[GoalDiff_list.index(j)], 'Ranking'] = j + duplication1-1
            elif GoalDiff_list.count(j) >= 2:
                idx = -1
                Duplication_list2 = []
                for cnt in range(GoalDiff_list.count(j)):
                    idx = GoalDiff_list.index(j, idx+1)
                    Duplication_list2.append(GoalDiff_index[idx])
                RoundRobin.loc[Duplication_list2, 'Ranking'] = j + duplication1-1
                RoundRobin_duplication2 = RoundRobin.loc[Duplication_list2]
                Score_rank = RoundRobin_duplication2['Score'].rank(ascending=False, method='min')
                Score_list = Score_rank.tolist()
                Score_index = Score_rank.index.tolist()
                for k in range(1,len(Score_list)+1):
                    if Score_list.count(k) == 1:
                        RoundRobin.at[Score_index[Score_list.index(k)], 'Ranking'] = k + j-1 + duplication1-1
    
    
    return RoundRobin

In [4]:
#Score_list        = ['4,0', '3,0', '2,0', '1,0', '0,0', '0,1', '0,2', '0,3', '0,4']
#Score_inverselist = ['0,4', '0,3', '0,2', '0,1', '0,0', '1,0', '2,0', '3,0', '4,0']
#Score_list        = ['4,0'
#                     , '4,1', '3,0'
#                     , '4,2', '3,1', '2,0'
#                     , '4,3', '3,2', '2,1', '1,0'
#                     , '4,4', '3,3', '2,2', '1,1', '0,0'
#                     , '3,4', '2,3', '1,2', '0,1'
#                     , '2,4', '1,3', '0,2'
#                     , '1,4', '0,3'
#                    , '0,4']
#Score_inverselist = ['0,4'
#                     , '1,4', '0,3'
#                     , '2,4', '1,3', '0,2'
#                     , '3,4', '2,3', '1,2', '0,1'
#                     , '4,4', '3,3', '2,2', '1,1', '0,0'
#                     , '4,3', '3,2', '2,1', '1,0'
#                     , '4,2', '3,1', '2,0'
#                     , '4,1', '3,0'
#                     , '4,0']
Score_list        = ['7,0'
                     , '7,1', '6,0'
                     , '7,2', '6,1', '5,0'
                     , '7,3', '6,2', '5,1', '4,0'
                     , '7,4', '6,3', '5,2', '4,1', '3,0'
                     , '7,5', '6,4', '5,3', '4,2', '3,1', '2,0'
                     , '7,6', '6,5', '5,4', '4,3', '3,2', '2,1', '1,0'
                     , '7,7', '6,6', '5,5', '4,4', '3,3', '2,2', '1,1', '0,0'
                     , '6,7', '5,6', '4,5', '3,4', '2,3', '1,2', '0,1'
                     , '5,7', '4,6', '3,5', '2,4', '1,3', '0,2'
                     , '4,7', '3,6', '2,5', '1,4', '0,3'
                     , '3,7', '2.6', '1,5', '0,4'
                     , '2,7', '1,6', '0,5'
                     , '1,7', '0,6'
                     , '0,7']
Score_inverselist = ['0,7'
                     , '1,7', '0,6'
                     , '2,7', '1,6', '0,5'
                     , '3,7', '2.6', '1,5', '0,4'
                     , '4,7', '3,6', '2,5', '1,4', '0,3'
                     , '5,7', '4,6', '3,5', '2,4', '1,3', '0,2'
                     , '6,7', '5,6', '4,5', '3,4', '2,3', '1,2', '0,1'
                     , '7,7', '6,6', '5,5', '4,4', '3,3', '2,2', '1,1', '0,0'
                     , '7,6', '6,5', '5,4', '4,3', '3,2', '2,1', '1,0'
                     , '7,5', '6,4', '5,3', '4,2', '3,1', '2,0'
                     , '7,4', '6,3', '5,2', '4,1', '3,0'
                     , '7,3', '6,2', '5,1', '4,0'
                     , '7,2', '6,1', '5,0'
                     , '7,1', '6,0'
                     , '7,0']

In [5]:
def RankingProbability(RoundRobin):
    Team_list = RoundRobin.index.tolist()
    #Score_list = ['5,0', '4,0', '3,0', '2,0', '1,0', '0,0', '0,1', '0,2', '0,3', '0,4', '0,5']
    #Score_list = ['1,0', '0,0', '0,1']
    # 7パターンで420秒(7分), 1秒で約240回
    # 予測：6試合分　9パターンで37分,11パターンで123分
    # 予測：4試合分　36パターンで42分x8=5時間34分, 25パターンで8分30秒x8=68分
    
    
    X = np.array(range(len(Score_list)))
    
    Ranking_num = pd.DataFrame(0, columns=range(1,5), index=Team_list)
    Ranking_probability = pd.DataFrame(columns=range(1,5), index=Team_list)
    print("start : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
    continueSwitch1 = 0
    for i1 in X:
        if continueSwitch1 == 1: continue
        #print("0,1(" + str(i1) + ") : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
        RoundRobin1 = RoundRobin.copy()
        if RoundRobin1.iat[0,1] is float(np.nan):
            RoundRobin1.iat[0,1] = Score_list[i1]
            RoundRobin1.iat[1,0] = Score_inverselist[i1]
        else: continueSwitch1 = 1
        continueSwitch2 = 0
        for i2 in X:
            if continueSwitch2 == 1: continue
            #print("0,2 : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
            RoundRobin2 = RoundRobin1.copy()
            if RoundRobin2.iat[0,2] is float(np.nan):
                RoundRobin2.iat[0,2] = Score_list[i2]
                RoundRobin2.iat[2,0] = Score_inverselist[i2]
            else: continueSwitch2 = 1
            continueSwitch3 = 0
            for i3 in X:
                if continueSwitch3 == 1: continue
                #print("0,3 : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
                RoundRobin3 = RoundRobin2.copy()
                if RoundRobin3.iat[0,3] is float(np.nan):
                    RoundRobin3.iat[0,3] = Score_list[i3]
                    RoundRobin3.iat[3,0] = Score_inverselist[i3]
                else: continueSwitch3 = 1
                continueSwitch4 = 0
                for i4 in X:
                    if continueSwitch4 == 1: continue
                    #print("1,2 : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
                    RoundRobin4 = RoundRobin3.copy()
                    if RoundRobin4.iat[1,2] is float(np.nan):
                        RoundRobin4.iat[1,2] = Score_list[i4]
                        RoundRobin4.iat[2,1] = Score_inverselist[i4]
                    else: continueSwitch4 = 1
                    continueSwitch5 = 0
                    for i5 in X:
                        if continueSwitch5 == 1: continue
                        #print("1,3 : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
                        RoundRobin5 = RoundRobin4.copy()
                        if RoundRobin5.iat[1,3] is float(np.nan):
                            RoundRobin5.iat[1,3] = Score_list[i5]
                            RoundRobin5.iat[3,1] = Score_inverselist[i5]
                        else: continueSwitch5 = 1
                        continueSwitch6 = 0
                        for i6 in X:
                            if continueSwitch6 == 1: continue
                            #print("2,3 : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
                            RoundRobin6 = RoundRobin5.copy()
                            if RoundRobin6.iat[2,3] is float(np.nan):
                                RoundRobin6.iat[2,3] = Score_list[i6]
                                RoundRobin6.iat[3,2] = Score_inverselist[i6]
                            else: continueSwitch6 = 1
                                
                            RoundRobin_rank = Ranking(RoundRobin6.copy())
                            
                            for j in range(1,5):
                                for k in range(0,4):
                                    if RoundRobin_rank.at[Team_list[k], 'Ranking'] == j:
                                        Ranking_num.at[Team_list[k], j] += 1
                            
    
    for k in range(0,4):
        patternNum = Ranking_num.loc[Team_list[k], :].sum()
        for j in range(1,5):
            Ranking_probability.at[Team_list[k], j] = str((Ranking_num.at[Team_list[k], j]/patternNum*100).round(1)) + '%'
        Ranking_probability.at[Team_list[k], 'Tournament'] = str(((Ranking_num.at[Team_list[k], 1]+Ranking_num.at[Team_list[k], 2])/patternNum*100).round(1)) + '%'
        
    return Ranking_num, Ranking_probability

# Group_A

In [6]:
Group_A_Team = DFTeamData.loc[DFTeamData['Group'] == 'A']
Group_A_Team_list = Group_A_Team['Team'].tolist()
Group_A_RoundRobin = pd.DataFrame(columns=Group_A_Team_list, index=Group_A_Team_list)
for team in Group_A_Team_list:
    Group_A_RoundRobin.at[team, team] = "-"

In [7]:
Group_A_RoundRobin.iat[0, 1], Group_A_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_A_RoundRobin.iat[2, 3], Group_A_RoundRobin.iat[3, 2] = '0,2' , '2,0'

Group_A_RoundRobin.iat[0, 2], Group_A_RoundRobin.iat[2, 0] = '1,3' , '3,1'
Group_A_RoundRobin.iat[1, 3], Group_A_RoundRobin.iat[3, 1] = '1,1' , '1,1'

#Group_A_RoundRobin.iat[0, 3], Group_A_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_A_RoundRobin.iat[1, 2], Group_A_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_A_RoundRobin)

,Qatar,Ecuador,Senegal,Netherlands,WinningPoints,Score,Conceded,GoalDiff,Ranking
Qatar,-,"0,2","1,3",NaN,0.0,1.0,5.0,-4.0,4.0
Ecuador,"2,0",-,NaN,"1,1",4.0,3.0,1.0,2.0,1.0
Senegal,"3,1",NaN,-,"0,2",3.0,3.0,3.0,0.0,3.0
Netherlands,NaN,"1,1","2,0",-,4.0,3.0,1.0,2.0,1.0


In [8]:
Ranking_num_A, Ranking_probability_A = RankingProbability(Group_A_RoundRobin.copy())

start : 2022.11.29 17:45:41


In [9]:
Ranking_num_A

,1,2,3,4
Qatar,0,0,588,3508
Ecuador,1674,1036,1386,0
Senegal,1008,929,1587,572
Netherlands,1450,2129,517,0


In [10]:
Ranking_probability_A

,1,2,3,4,Tournament
Qatar,0.0%,0.0%,14.4%,85.6%,0.0%
Ecuador,40.9%,25.3%,33.8%,0.0%,66.2%
Senegal,24.6%,22.7%,38.7%,14.0%,47.3%
Netherlands,35.4%,52.0%,12.6%,0.0%,87.4%


# Group_B

In [11]:
Group_B_Team = DFTeamData.loc[DFTeamData['Group'] == 'B']
Group_B_Team_list = Group_B_Team['Team'].tolist()
Group_B_RoundRobin = pd.DataFrame(columns=Group_B_Team_list, index=Group_B_Team_list)
for team in Group_B_Team_list:
    Group_B_RoundRobin.at[team, team] = "-"

In [12]:
Group_B_RoundRobin.iat[0, 1], Group_B_RoundRobin.iat[1, 0] = '6,2' , '2,6'
Group_B_RoundRobin.iat[2, 3], Group_B_RoundRobin.iat[3, 2] = '1,1' , '1,1'

Group_B_RoundRobin.iat[0, 2], Group_B_RoundRobin.iat[2, 0] = '0,0' , '0,0'
Group_B_RoundRobin.iat[1, 3], Group_B_RoundRobin.iat[3, 1] = '2,0' , '0,2'

#Group_B_RoundRobin.iat[0, 3], Group_B_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_B_RoundRobin.iat[1, 2], Group_B_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_B_RoundRobin)

,England,Iran,USA,Wales,WinningPoints,Score,Conceded,GoalDiff,Ranking
England,-,"6,2","0,0",NaN,4.0,6.0,2.0,4.0,1.0
Iran,"2,6",-,NaN,"2,0",3.0,4.0,6.0,-2.0,2.0
USA,"0,0",NaN,-,"1,1",2.0,1.0,1.0,0.0,3.0
Wales,NaN,"0,2","1,1",-,1.0,1.0,3.0,-2.0,4.0


In [13]:
Ranking_num_B, Ranking_probability_B = RankingProbability(Group_B_RoundRobin.copy())

start : 2022.11.29 17:45:48


In [14]:
Ranking_probability_B

,1,2,3,4,Tournament
England,53.2%,34.0%,12.7%,0.0%,87.3%
Iran,24.6%,26.6%,29.6%,19.1%,51.2%
USA,20.5%,23.3%,28.2%,28.0%,43.8%
Wales,1.8%,16.0%,29.5%,52.7%,17.8%


# Group_C

In [15]:
Group_C_Team = DFTeamData.loc[DFTeamData['Group'] == 'C']
Group_C_Team_list = Group_C_Team['Team'].tolist()
Group_C_RoundRobin = pd.DataFrame(columns=Group_C_Team_list, index=Group_C_Team_list)
for team in Group_C_Team_list:
    Group_C_RoundRobin.at[team, team] = "-"

In [16]:
Group_C_RoundRobin.iat[0, 1], Group_C_RoundRobin.iat[1, 0] = '1,2' , '2,1'
Group_C_RoundRobin.iat[2, 3], Group_C_RoundRobin.iat[3, 2] = '0,0' , '0,0'

Group_C_RoundRobin.iat[0, 2], Group_C_RoundRobin.iat[2, 0] = '2,0' , '0,2'
Group_C_RoundRobin.iat[1, 3], Group_C_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_C_RoundRobin.iat[0, 3], Group_C_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_C_RoundRobin.iat[1, 2], Group_C_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_C_RoundRobin)

,Argentina,SaudiArabia,Mexico,Poland,WinningPoints,Score,Conceded,GoalDiff,Ranking
Argentina,-,"1,2","2,0",NaN,3.0,3.0,2.0,1.0,2.0
SaudiArabia,"2,1",-,NaN,"0,2",3.0,2.0,3.0,-1.0,3.0
Mexico,"0,2",NaN,-,"0,0",1.0,0.0,2.0,-2.0,4.0
Poland,NaN,"2,0","0,0",-,4.0,2.0,0.0,2.0,1.0


In [17]:
Ranking_num_C, Ranking_probability_C = RankingProbability(Group_C_RoundRobin.copy())

start : 2022.11.29 17:45:54


In [18]:
Ranking_probability_C

,1,2,3,4,Tournament
Argentina,39.8%,8.7%,47.4%,4.0%,48.6%
SaudiArabia,9.7%,42.0%,8.8%,39.5%,51.7%
Mexico,0.0%,35.8%,7.9%,56.2%,35.8%
Poland,50.8%,13.7%,35.5%,0.0%,64.5%


# Group_D

In [19]:
Group_D_Team = DFTeamData.loc[DFTeamData['Group'] == 'D']
Group_D_Team_list = Group_D_Team['Team'].tolist()
Group_D_RoundRobin = pd.DataFrame(columns=Group_D_Team_list, index=Group_D_Team_list)
for team in Group_D_Team_list:
    Group_D_RoundRobin.at[team, team] = "-"

In [20]:
Group_D_RoundRobin.iat[0, 1], Group_D_RoundRobin.iat[1, 0] = '4,1' , '1,4'
Group_D_RoundRobin.iat[2, 3], Group_D_RoundRobin.iat[3, 2] = '0,0' , '0,0'

Group_D_RoundRobin.iat[0, 2], Group_D_RoundRobin.iat[2, 0] = '2,1' , '1,2'
Group_D_RoundRobin.iat[1, 3], Group_D_RoundRobin.iat[3, 1] = '1,0' , '0,1'

#Group_D_RoundRobin.iat[0, 3], Group_D_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_D_RoundRobin.iat[1, 2], Group_D_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_D_RoundRobin)

,France,Australia,Denmark,Tunisia,WinningPoints,Score,Conceded,GoalDiff,Ranking
France,-,"4,1","2,1",NaN,6.0,6.0,2.0,4.0,1.0
Australia,"1,4",-,NaN,"1,0",3.0,2.0,4.0,-2.0,2.0
Denmark,"1,2",NaN,-,"0,0",1.0,1.0,2.0,-1.0,3.0
Tunisia,NaN,"0,1","0,0",-,1.0,0.0,1.0,-1.0,4.0


In [21]:
Ranking_num_D, Ranking_probability_D = RankingProbability(Group_D_RoundRobin.copy())

start : 2022.11.29 17:46:00


In [22]:
Ranking_probability_D

,1,2,3,4,Tournament
France,91.9%,8.1%,0.0%,0.0%,100.0%
Australia,8.5%,42.3%,30.1%,19.1%,50.8%
Denmark,0.0%,35.0%,25.6%,39.4%,35.0%
Tunisia,0.0%,14.7%,44.4%,40.9%,14.7%


# Group_E

In [23]:
Group_E_Team = DFTeamData.loc[DFTeamData['Group'] == 'E']
Group_E_Team_list = Group_E_Team['Team'].tolist()
Group_E_RoundRobin = pd.DataFrame(columns=Group_E_Team_list, index=Group_E_Team_list)
for team in Group_E_Team_list:
    Group_E_RoundRobin.at[team, team] = "-"

In [24]:
Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '1,1' , '1,1'
Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '1,0' , '0,1'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)

,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1",NaN,4.0,8.0,1.0,7.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,NaN,"0,1","2,1",-,3.0,2.0,2.0,0.0,2.0


In [25]:
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())

start : 2022.11.29 17:46:06


In [26]:
Ranking_probability_E

,1,2,3,4,Tournament
Spain,50.8%,27.0%,22.2%,0.0%,77.8%
CostaRica,5.6%,43.6%,7.2%,43.6%,49.2%
Germany,0.0%,27.5%,16.3%,56.2%,27.5%
Japan,43.6%,2.4%,53.9%,0.1%,46.0%


# Group_F

In [27]:
Group_F_Team = DFTeamData.loc[DFTeamData['Group'] == 'F']
Group_F_Team_list = Group_F_Team['Team'].tolist()
Group_F_RoundRobin = pd.DataFrame(columns=Group_F_Team_list, index=Group_F_Team_list)
for team in Group_F_Team_list:
    Group_F_RoundRobin.at[team, team] = "-"

In [28]:
Group_F_RoundRobin.iat[0, 1], Group_F_RoundRobin.iat[1, 0] = '1,0' , '0,1'
Group_F_RoundRobin.iat[2, 3], Group_F_RoundRobin.iat[3, 2] = '0,0' , '0,0'

Group_F_RoundRobin.iat[0, 2], Group_F_RoundRobin.iat[2, 0] = '0,2' , '2,0'
Group_F_RoundRobin.iat[1, 3], Group_F_RoundRobin.iat[3, 1] = '1,4' , '4,1'

#Group_F_RoundRobin.iat[0, 3], Group_F_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_F_RoundRobin.iat[1, 2], Group_F_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_F_RoundRobin)

,Belgium,Canada,Morocco,Croatia,WinningPoints,Score,Conceded,GoalDiff,Ranking
Belgium,-,"1,0","0,2",NaN,3.0,1.0,2.0,-1.0,3.0
Canada,"0,1",-,NaN,"1,4",0.0,1.0,5.0,-4.0,4.0
Morocco,"2,0",NaN,-,"0,0",4.0,2.0,0.0,2.0,2.0
Croatia,NaN,"4,1","0,0",-,4.0,4.0,1.0,3.0,1.0


In [29]:
Ranking_num_F, Ranking_probability_F = RankingProbability(Group_F_RoundRobin.copy())

start : 2022.11.29 17:46:12


In [30]:
Ranking_probability_F

,1,2,3,4,Tournament
Belgium,24.6%,21.5%,37.0%,16.9%,46.1%
Canada,0.0%,0.0%,17.1%,82.9%,0.0%
Morocco,30.6%,54.1%,15.4%,0.0%,84.6%
Croatia,45.2%,24.5%,30.3%,0.0%,69.7%


# Group_G

In [31]:
Group_G_Team = DFTeamData.loc[DFTeamData['Group'] == 'G']
Group_G_Team_list = Group_G_Team['Team'].tolist()
Group_G_RoundRobin = pd.DataFrame(columns=Group_G_Team_list, index=Group_G_Team_list)
for team in Group_G_Team_list:
    Group_G_RoundRobin.at[team, team] = "-"

In [32]:
Group_G_RoundRobin.iat[0, 1], Group_G_RoundRobin.iat[1, 0] = '2,0' , '0,2'
Group_G_RoundRobin.iat[2, 3], Group_G_RoundRobin.iat[3, 2] = '1,0' , '0,1'

Group_G_RoundRobin.iat[0, 2], Group_G_RoundRobin.iat[2, 0] = '1,0' , '0,1'
Group_G_RoundRobin.iat[1, 3], Group_G_RoundRobin.iat[3, 1] = '3,3' , '3,3'

#Group_G_RoundRobin.iat[0, 3], Group_G_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_G_RoundRobin.iat[1, 2], Group_G_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_G_RoundRobin)

,Brazil,Serbia,Switzerland,Cameroon,WinningPoints,Score,Conceded,GoalDiff,Ranking
Brazil,-,"2,0","1,0",NaN,6.0,3.0,0.0,3.0,1.0
Serbia,"0,2",-,NaN,"3,3",1.0,3.0,5.0,-2.0,4.0
Switzerland,"0,1",NaN,-,"1,0",3.0,1.0,1.0,0.0,2.0
Cameroon,NaN,"3,3","0,1",-,1.0,3.0,4.0,-1.0,3.0


In [33]:
Ranking_num_G, Ranking_probability_G = RankingProbability(Group_G_RoundRobin.copy())

start : 2022.11.29 17:46:18


In [34]:
Ranking_probability_G

,1,2,3,4,Tournament
Brazil,83.7%,16.3%,0.0%,0.0%,100.0%
Serbia,0.0%,31.6%,24.6%,43.8%,31.6%
Switzerland,16.6%,34.5%,29.7%,19.1%,51.1%
Cameroon,0.0%,17.9%,45.6%,36.6%,17.9%


# Group_H

In [35]:
Group_H_Team = DFTeamData.loc[DFTeamData['Group'] == 'H']
Group_H_Team_list = Group_H_Team['Team'].tolist()
Group_H_RoundRobin = pd.DataFrame(columns=Group_H_Team_list, index=Group_H_Team_list)
for team in Group_H_Team_list:
    Group_H_RoundRobin.at[team, team] = "-"

In [36]:
Group_H_RoundRobin.iat[0, 1], Group_H_RoundRobin.iat[1, 0] = '3,2' , '2,3'
Group_H_RoundRobin.iat[2, 3], Group_H_RoundRobin.iat[3, 2] = '0,0' , '0,0'

Group_H_RoundRobin.iat[0, 2], Group_H_RoundRobin.iat[2, 0] = '2,0' , '0,2'
Group_H_RoundRobin.iat[1, 3], Group_H_RoundRobin.iat[3, 1] = '3,2' , '2,3'

#Group_H_RoundRobin.iat[0, 3], Group_H_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_H_RoundRobin.iat[1, 2], Group_H_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_H_RoundRobin)

,Portugal,Ghana,Uruguay,Korea,WinningPoints,Score,Conceded,GoalDiff,Ranking
Portugal,-,"3,2","2,0",NaN,6.0,5.0,2.0,3.0,1.0
Ghana,"2,3",-,NaN,"3,2",3.0,5.0,5.0,0.0,2.0
Uruguay,"0,2",NaN,-,"0,0",1.0,0.0,2.0,-2.0,4.0
Korea,NaN,"2,3","0,0",-,1.0,2.0,3.0,-1.0,3.0


In [37]:
Ranking_num_H, Ranking_probability_H = RankingProbability(Group_H_RoundRobin.copy())

start : 2022.11.29 17:46:24


In [38]:
Ranking_probability_H

,1,2,3,4,Tournament
Portugal,83.1%,16.9%,0.0%,0.0%,100.0%
Ghana,17.1%,34.8%,29.0%,19.1%,51.9%
Uruguay,0.0%,30.6%,24.7%,44.8%,30.6%
Korea,0.0%,18.0%,46.2%,35.8%,18.0%


# Group_XXX

Group_XXX_Team = DFTeamData.loc[DFTeamData['Group'] == 'XXX']
Group_XXX_Team_list = Group_XXX_Team['Team'].tolist()
Group_XXX_RoundRobin = pd.DataFrame(columns=Group_XXX_Team_list, index=Group_XXX_Team_list)
for team in Group_XXX_Team_list:
    Group_XXX_RoundRobin.at[team, team] = "-"

Group_XXX_RoundRobin.iat[0, 1], Group_XXX_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_XXX_RoundRobin.iat[2, 3], Group_XXX_RoundRobin.iat[3, 2] = '0,2' , '2,0'

#Group_XXX_RoundRobin.iat[0, 2], Group_XXX_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_XXX_RoundRobin.iat[1, 3], Group_XXX_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_XXX_RoundRobin.iat[0, 3], Group_XXX_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_XXX_RoundRobin.iat[1, 2], Group_XXX_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_XXX_RoundRobin)

Ranking_num_XXX, Ranking_probability_XXX = RankingProbability(Group_XXX_RoundRobin.copy())

Ranking_probability_XXX